<a href="https://colab.research.google.com/github/hrmufid/Mini-Project-3/blob/main/Mini_Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import shap

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# initialize random state
random_state = 42

# set plot style for whole notebook
sns.set_style('darkgrid')

In [ ]:
# IMPORT DATA

drive.mount('/content/drive')

df = pd.read_csv("https://drive.google.com/file/d/19TUlAkMBRQi4MKfimeYBxCrFSeYk0ZGr/view?usp=drive_link")
df.head()

NameError: ignored